In [251]:
from pprint import pprint as pp
from tqdm import tqdm as tq
from datetime import date
import pandas as pd
import numpy as np
import json

data_path = 'H:/linewalks/' # 필요한 csv 파일의 경로 설정
df_concept = pd.read_csv(data_path + 'concept.csv', encoding = 'utf-8')
df_condition_occurrence = pd.read_csv(data_path + 'condition_occurrence.csv', encoding='utf-8')
df_death = pd.read_csv(data_path + 'death.csv',encoding='utf-8' )
df_drug_exposure = pd.read_csv(data_path + 'drug_exposure.csv', encoding='utf-8')
df_person = pd.read_csv(data_path + 'person.csv',encoding='utf-8')
df_visit_occurrence = pd.read_csv(data_path + 'visit_occurrence.csv',encoding='utf-8')

# 내원일수를 계산해주는 함수
def calDate_list(de_list,ds_list) :
    res = []
    for i in range(len(de_list)) :
        ds = list(map(int, ds_list[i].split('-')))
        de = list(map(int, de_list[i].split('-')))
        ds = date(ds[0], ds[1], ds[2])
        de = date(de[0], de[1], de[2])
        delta = de - ds
        res.append(delta.days)
    return(res)

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [253]:
# 자주 쓰게되는 DataFrame의 Col
_p = df_person['person_id']
death_p = df_death['person_id']
Dead_p = df_person[np.in1d(_p,death_p)]
Alive_p = df_person[~np.in1d(_p,death_p)]

In [499]:
# 카테고리 a : 1. 사망자 (152명) 중 2020-3월이 사망날짜인 사람 (27명) (17.76%)
print(len([i for i in df_death['death_date'] if str(i).split('-')[0] == '2020' and str(i).split('-')[1] == '03']) / len(death_p) * 100,'%')
# 해당 데이터는 사망률과 관련이 없다.

17.763157894736842 %


In [627]:
# 카테고리 b : 1 성별
# 사망자 152명과 생존자 848명(랜덤으로 샘플링한 데이터) 의 성별 상관관계
alive_gender_S = pd.DataFrame({'gender' : [1 if i==8532 else 0 for i in df_person[~np.in1d(_p, death_p)]['gender_concept_id']],'isdead':[0 for _ in range(len(df_person[~np.in1d(_p,death_p)]))]}).sample(n=len(df_person[np.in1d(_p, death_p)]['gender_concept_id']))
dead_gender = pd.DataFrame({'gender' :[1 if i==8532 else 0 for i in df_person[np.in1d(_p, death_p)]['gender_concept_id']], 'isdead' : [1 for _ in range(len(df_person[np.in1d(_p,death_p)]))]})

alive_gender_S = alive_gender_S.reset_index().drop(columns=['index'])
dead_gender = dead_gender.reset_index().drop(columns=['index'])
df_gender = pd.concat([alive_gender_S,dead_gender]).reset_index().drop(columns=['index'])

# 상관계수의 절대값이 0.2 이하이므로, 상관관계가 있다고 보기 힘들다
df_gender.corr(method='pearson')

,gender,isdead
gender,1.000000,-0.172553
isdead,-0.172553,1.000000


In [629]:
# 카테고리 b : 2 인종
# 사망자 152명과 생존자 848명(랜덤으로 샘플링한 데이터) 의 인종 상관관계
alive_race_S = pd.DataFrame(
    {'isAsian' : [1 if i == 'asian' else 0 for i in df_person[~np.in1d(_p, death_p)]['race_source_value']],
     'isBlack' : [1 if i == 'black' else 0 for i in df_person[~np.in1d(_p, death_p)]['race_source_value']],
     'isWhite' : [1 if i == 'white' else 0 for i in df_person[~np.in1d(_p, death_p)]['race_source_value']],
     'isdead':[0 for _ in range(len(df_person[~np.in1d(_p,death_p)]))]}).sample(n=len(df_person[np.in1d(_p, death_p)]['race_source_value']))
dead_race = pd.DataFrame(
    {'isAsian' : [1 if i == 'asian' else 0 for i in df_person[np.in1d(_p, death_p)]['race_source_value']],
     'isBlack' : [1 if i == 'black' else 0 for i in df_person[np.in1d(_p, death_p)]['race_source_value']],
     'isWhite' : [1 if i == 'white' else 0 for i in df_person[np.in1d(_p, death_p)]['race_source_value']],
     'isdead' : [1 for _ in range(len(df_person[np.in1d(_p,death_p)]))]})

alive_race_S = alive_race_S.reset_index().drop(columns=['index'])
dead_race = dead_race.reset_index().drop(columns=['index'])
df_race = pd.concat([alive_race_S,dead_race])

# 상관계수의 절대값이 0.2 이하이므로, 상관관계가 있다고 보기 힘들다
df_race.corr(method='pearson')

,isAsian,isBlack,isWhite,isdead
isAsian,1.000000,-0.080611,-0.611484,0.013589
isBlack,-0.080611,1.000000,-0.739389,0.011563
isWhite,-0.611484,-0.739389,1.000000,-0.018359
isdead,0.013589,0.011563,-0.018359,1.000000


In [626]:
# 카테고리 b : 3 나이
# 사망자 152명과 생존자 848명(랜덤으로 샘플링한 데이터) 의 상관관계
alive_age_S = pd.DataFrame(
    {'age' : [2020 - i for i in df_person[~np.in1d(_p,death_p)]['year_of_birth']],
     'isdead':[0 for _ in range(len(df_person[~np.in1d(_p,death_p)]))]}).sample(n=len(df_person[np.in1d(_p,death_p)]['year_of_birth']))
dead_age = pd.DataFrame(
    {'age' :[2020 - i for i in df_person[np.in1d(_p,death_p)]['year_of_birth']],
     'isdead' : [1 for _ in range(len(df_person[np.in1d(_p,death_p)]))]})

# 나이 계산은 만나이 에서 생일을 고려하지 않은 나이로 계산

alive_age_S = alive_age_S.reset_index().drop(columns=['index'])
dead_age = dead_age.reset_index().drop(columns=['index'])
df_age = pd.concat([alive_age_S,dead_age])

# 0.62로 나오는걸로 봐서 양적 상관관계로 볼 수 있다.
df_age.corr(method='pearson')

,age,isdead
age,1.000000,0.610535
isdead,0.610535,1.000000


In [654]:
# 카테고리 c : 1/2/3 입원/외래/응급
# 사망자 152명과 생존자 848명(랜덤으로 샘플링한 데이터) 의 인종 상관관계
alive_visit_S = pd.DataFrame(
    {'입원' : [1 if i == 9201 else 0 for i in df_visit_occurrence[~np.in1d(df_visit_occurrence['person_id'],death_p)]['visit_concept_id']],
     '외래' : [1 if i == 9202 else 0 for i in df_visit_occurrence[~np.in1d(df_visit_occurrence['person_id'],death_p)]['visit_concept_id']],
     '응급' : [1 if i == 9203 else 0 for i in df_visit_occurrence[~np.in1d(df_visit_occurrence['person_id'],death_p)]['visit_concept_id']],
     'isdead':[0 for _ in range(len(df_visit_occurrence[~np.in1d(df_visit_occurrence['person_id'],death_p)]['visit_concept_id']))]}).sample(n=len(df_visit_occurrence[np.in1d(df_visit_occurrence['person_id'],death_p)]['visit_concept_id']))
dead_visit = pd.DataFrame(
    {'입원' : [1 if i == 9201 else 0 for i in df_visit_occurrence[np.in1d(df_visit_occurrence['person_id'],death_p)]['visit_concept_id']],
     '외래' : [1 if i == 9202 else 0 for i in df_visit_occurrence[np.in1d(df_visit_occurrence['person_id'],death_p)]['visit_concept_id']],
     '응급' : [1 if i == 9203 else 0 for i in df_visit_occurrence[np.in1d(df_visit_occurrence['person_id'],death_p)]['visit_concept_id']],
     'isdead' : [1 for _ in range(len(df_visit_occurrence[np.in1d(df_visit_occurrence['person_id'],death_p)]['visit_concept_id']))]})

alive_visit_S = alive_visit_S.reset_index().drop(columns=['index'])
dead_visit = dead_visit.reset_index().drop(columns=['index'])
df_visit = pd.concat([alive_visit_S,dead_visit])

# 상관계수의 절대값이 0.2 이하이므로, 상관관계가 있다고 보기 힘들다
df_visit.corr(method='pearson')

,입원,외래,응급,isdead
입원,1.000000,-0.537516,-0.064527,0.106883
외래,-0.537516,1.000000,-0.806812,-0.076808
응급,-0.064527,-0.806812,1.000000,0.016011
isdead,0.106883,-0.076808,0.016011,1.000000


In [658]:
deadPersonVisitEndDateList = df_visit_occurrence[np.in1d(df_visit_occurrence['person_id'], death_p)]['visit_end_date'].tolist()
deadPersonVisitStartDateList =  df_visit_occurrence[np.in1d(df_visit_occurrence['person_id'], death_p)]['visit_start_date'].tolist()
alivePersonVisitEndDateList = df_visit_occurrence[~np.in1d(df_visit_occurrence['person_id'], death_p)]['visit_end_date'].tolist()
alivePersonVisitStartDateList =  df_visit_occurrence[~np.in1d(df_visit_occurrence['person_id'], death_p)]['visit_start_date'].tolist()

# 카테고리 c : 4 내원일수
# 사망자 152명과 생존자 848명(랜덤으로 샘플링한 데이터) 의 내원일수 상관관계
alive_visit_day_S = pd.DataFrame({'day' : [i for i in calDate_list( alivePersonVisitEndDateList , alivePersonVisitStartDateList )],
                                  'isdead':[0 for _ in range(len(calDate_list( alivePersonVisitEndDateList , alivePersonVisitStartDateList )))]}).sample(n=len(calDate_list( deadPersonVisitEndDateList , deadPersonVisitStartDateList )))
dead_visit_day = pd.DataFrame({'day' :[i for i in calDate_list( deadPersonVisitEndDateList , deadPersonVisitStartDateList )], 
                               'isdead' : [1 for _ in range(len(calDate_list( deadPersonVisitEndDateList , deadPersonVisitStartDateList )))]})

alive_visit_day_S = alive_visit_day_S.reset_index().drop(columns=['index'])
dead_visit_day = dead_visit_day.reset_index().drop(columns=['index'])
df_visit_day = pd.concat([alive_visit_day_S,dead_visit_day])

# 상관계수의 절대값이 0.2 이하이므로, 상관관계가 있다고 보기 힘들다
df_visit_day.corr(method='pearson')

,day,isdead
day,1.000000,0.034667
isdead,0.034667,1.000000


### 이 아래는 상관계수를 이용하여 구한값이 아닌, 사망자/생존자 대비 변수값의 퍼센티지를 구하는 코드

In [661]:
# 카테고리 b : 1 사망자(152명) 중 남성(99명) (65.13%)
print(' 사망자 중, 성별 남성 비율 : ', df_person[np.in1d(_p, death_p)]['gender_concept_id'].value_counts()[8507] / len(death_p) * 100 ,'%')

print(' 생존자 중, 성별 남성 비율 : ', df_person[~np.in1d(_p, death_p)]['gender_concept_id'].value_counts()[8507] / len(_p) * 100 ,'%')

 사망자 중, 성별 남성 비율 :  65.13157894736842 %
 생존자 중, 성별 남성 비율 :  44.9 %


In [338]:
# 카테고리 b : 2 사망자(152명) 중 남성(99명) (65.13%)
print(' 사망자 중, 인종 백인 비율 : ', df_person[np.in1d(_p, death_p)]['race_concept_id'].value_counts()[8527] / len(death_p) * 100 ,'%')

print(' 생존자 중, 인종 백인 비율 : ', df_person[~np.in1d(_p, death_p)]['race_concept_id'].value_counts()[8527] / len(_p) * 100 ,'%')

 사망자 중, 인종 백인 비율 :  84.21052631578947 %
 생존자 중, 인종 백인 비율 :  71.7 %


In [277]:
#민족 컬럼에는 데이터가 없음
df_person['ethnicity_concept_id'].value_counts()

0    1000
Name: ethnicity_concept_id, dtype: int64

In [292]:
# 카테고리 b : 3 사망자 (152명) 중 70세 이상 96명 (63.15%)
D_age = pd.Series(sorted([2021 - i for i in df_person[np.in1d(_p,death_p)]['year_of_birth']])).tolist()
print(' 사망자 중, 나이가 70세 이상인 노인의 비율 : ',len([i for i in D_age if i > 70]) / len(death_p) * 100 ,'%')
# 카테고리 b : 3 생망자 (848명) 중 70세 이상 96명 (12.97%)
A_age = pd.Series(sorted([2021 - i for i in df_person[~np.in1d(_p,death_p)]['year_of_birth']])).tolist()
print(' 생존자 중, 나이가 70세 이상인 노인의 비율 : ',len([i for i in A_age if i > 70]) / len(Alive_p) * 100 ,'%')

 사망자 중, 나이가 70세 이상인 노인의 비율 :  63.1578947368421
 생존자 중, 나이가 70세 이상인 노인의 비율 :  12.971698113207546 %


In [301]:
# 카테고리 c : 1 사망자 (152명) 의 전체 진료 횟수 중 입원 횟수 
dead_person_visit = df_visit_occurrence[np.in1d(df_visit_occurrence['person_id'],death_p)]
alive_person_visit = df_visit_occurrence[~np.in1d(df_visit_occurrence['person_id'],death_p)]

print(' 사망자 중, 입원 횟수 비율 : ', dead_person_visit['visit_concept_id'].value_counts()[9201] / len(dead_person_visit) * 100,'%')
print(' 생존자 중, 입원 횟수 비율 : ', alive_person_visit['visit_concept_id'].value_counts()[9201] / len(alive_person_visit) * 100,'%')

 사망자 중, 입원 횟수 비율 :  6.246450880181715 %
 생존자 중, 입원 횟수 비율 :  2.0771988221738575 %


In [300]:
# 카테고리 c : 2 사망자 (152명) 의 전체 진료 횟수 중 외래 횟수
dead_person_visit = df_visit_occurrence[np.in1d(df_visit_occurrence['person_id'],death_p)]
alive_person_visit = df_visit_occurrence[~np.in1d(df_visit_occurrence['person_id'],death_p)]

print(' 사망자 중, 외래 횟수 비율 : ', dead_person_visit['visit_concept_id'].value_counts()[9202] / len(dead_person_visit) * 100,'%')
print(' 생존자 중, 외래 횟수 비율 : ', alive_person_visit['visit_concept_id'].value_counts()[9202] / len(alive_person_visit) * 100,'%')

 사망자 중, 외래 횟수 비율 :  84.46905167518456 %
 생존자 중, 외래 횟수 비율 :  89.94046856996543 %


In [299]:
# 카테고리 c : 3 사망자 (152명) 의 전체 진료 횟수 중 응급 횟수 
dead_person_visit = df_visit_occurrence[np.in1d(df_visit_occurrence['person_id'],death_p)]
alive_person_visit = df_visit_occurrence[~np.in1d(df_visit_occurrence['person_id'],death_p)]

print(' 사망자 중, 응급 횟수 비율 : ', dead_person_visit['visit_concept_id'].value_counts()[9203] / len(dead_person_visit) * 100,'%')
print(' 생존자 중, 응급 횟수 비율 : ', alive_person_visit['visit_concept_id'].value_counts()[9203] / len(alive_person_visit) * 100,'%')

 사망자 중, 응급 횟수 비율 :  9.28449744463373 %
 생존자 중, 응급 횟수 비율 :  7.982332607860709 %


In [324]:
# 카테고리 d : 1 사망자 (152명) 의 전체 진단 concept_id
dead_person_con = df_condition_occurrence[np.in1d(df_condition_occurrence['person_id'], death_p)]
print(' 사망자 중, concept_id 0 비율 : ', dead_person_con['condition_concept_id'].value_counts()[0] / len(dead_person_con) * 100,'%')
print(' 사망자 중, concept_id 40481087 비율 : ', dead_person_con['condition_concept_id'].value_counts()[40481087] / len(dead_person_con) * 100,'%')

# 카테고리 d : 1 생존자 (848명) 의 전체 진단 concept_id
alive_person_con = df_condition_occurrence[~np.in1d(df_condition_occurrence['person_id'], death_p)]
print(' 생존자 중, concept_id 0 비율 : ', alive_person_con['condition_concept_id'].value_counts()[0] / len(alive_person_con) * 100,'%')
print(' 생존자 중, concept_id 40481087 비율 : ', alive_person_con['condition_concept_id'].value_counts()[40481087] / len(alive_person_con) * 100,'%')

 사망자 중, concept_id 0 비율 :  9.953584321815368 %
 생존자 중, concept_id 0 비율 :  11.136096988658583 %
 사망자 중, concept_id 40481087 비율 :  8.45796802475503 %
 생존자 중, concept_id 40481087 비율 :  8.672272193977317 %


In [329]:
# 카테고리 d : 1 사망자 (152명) 의 진단 source_value
dead_person_con = df_condition_occurrence[np.in1d(df_condition_occurrence['person_id'], death_p)]
alive_person_con = df_condition_occurrence[~np.in1d(df_condition_occurrence['person_id'], death_p)]
print(' 사망자 중, condition_source_value 444814009 비율 : ', dead_person_con['condition_source_value'].value_counts()[444814009] / len(dead_person_con) * 100,'%')
print(' 생존자 중, condition_source_value 444814009 비율 : ', alive_person_con['condition_source_value'].value_counts()[444814009] / len(alive_person_con) * 100,'%')

 사망자 중, condition_source_value 444814009 비율 :  8.45796802475503 %
 생존자 중, condition_source_value 444814009 비율 :  8.672272193977317 %


In [336]:
# 카테고리 e : 1 사망자 (152명) 의 전체 진단 횟수 중 외래 진료 횟수 (63.15%) 총 진료 횟수 : 10566 
dead_drug_exposure = df_drug_exposure[np.in1d(df_drug_exposure['person_id'], death_p)]
alive_drug_exposure = df_drug_exposure[~np.in1d(df_drug_exposure['person_id'], death_p)]
print(' 사망자 중, 의약품 444814009(Influenza, seasonal..) 비율 : ', dead_drug_exposure['drug_concept_id'].value_counts()[40213154] / len(dead_drug_exposure) * 100,'%')
print(' 생존자 중, 의약품 444814009(Influenza, seasonal..) 비율 : ', alive_drug_exposure['drug_concept_id'].value_counts()[40213154] / len(alive_drug_exposure) * 100,'%')
#40213154    19078106    
print(' 사망자 중, 의약품 19078106(hydrochlorothiazide 25..) 비율 : ', dead_drug_exposure['drug_concept_id'].value_counts()[19078106] / len(dead_drug_exposure) * 100,'%')
print(' 생존자 중, 의약품 19078106(hydrochlorothiazide 25..) 비율 : ', alive_drug_exposure['drug_concept_id'].value_counts()[19078106] / len(alive_drug_exposure) * 100,'%')

 사망자 중, 의약품 444814009(Influenza, seasonal..) 비율 :  9.71457185778668 %
 생존자 중, 의약품 444814009(Influenza, seasonal..) 비율 :  20.420245398773005 %
 사망자 중, 의약품 19078106(hydrochlorothiazide 25..) 비율 :  9.285356606338079 %
 생존자 중, 의약품 19078106(hydrochlorothiazide 25..) 비율 :  7.273006134969324 %


### 사망률과 연관성이 높은 변수 순서 : b. 3(나이) > e. 1(의약품) > c. 1,2,3,4 = b.1,2 . (a. 1 사망일은 사망률과 관련이 없는 데이터) . 
#### 나이를 제외하곤 상관관계를 보이는 변수는 없는것으로 보입니다.